# FEM Modelling for STEDIN Transformer

### Including Eddy Current Effects

In this part, the conducitvity of the core material is modelled as $2.22 \times 10^{6}$ (grain-oriented Fe-Si steel) [1].

[1] Fiorillo, Fausto. Characterization and measurement of magnetic materials. Academic Press, 2004.

Note : In actual scenario, the conductivity is not this high because of laminated core. Therefore, the conductivity can be modelled as $0.1$.

## Import Packages

In [1]:
print(" ▸ Loading Packages .... \r")
start = time_ns()
using gmsh
using LinearAlgebra
using LaTeXStrings
using Plots
using CSV
using DataFrames
include("../../../../lib/Makie_Plots.jl");
using .Makie_Plots
include("../../../../lib/FEM_Tri_1e.jl");
using .FEM_Tri_1e
include("../../../../lib/Post_Process_Frequency.jl");
using .Post_Process_Frequency
include("../../../../lib/Mesh_Data_stedin.jl");
using .Mesh_Data_stedin
include("../../../../lib/Save_VTK.jl");
using .Save_VTK
elapsed = round((time_ns() - start)/10^9, digits=2)
println(" ✓ Packages loaded ("*string(elapsed)*" seconds)                               ")

 ✓ Packages loaded (52.47 seconds)                               


## Julia code

In [2]:
gmsh.initialize()

## Read mesh from file
print(" ▸ Reading mesh file .... \r")
start = time_ns()
gmsh.open("../../../../Geometry/mesh/stedin_transformer.msh")
elapsed = round((time_ns() - start)/10^9, digits=2)
println(" ✓ Mesh file loaded ("*string(elapsed)*" seconds)                               ")

print(" ▸ Generating required mesh data .... \r")
start = time_ns()
mesh_data = get_mesh_data_tri_1e(gmsh)
elapsed = round((time_ns() - start)/10^9, digits=2)
println(" ✓ Mesh data generated ("*string(elapsed)*" seconds)                               ")

print(" ▸ Setting initial parameters .... \r")
start = time_ns()
Ip = 0;       # Primary peak phase current
Is = 777.62;  # Secondary peak phase current
Np = 266;
Ns = 6;

omega = 2*pi*50;  # Frequency

# HV winding dimensions (all phases left/right are identical)
wwhv = 3e-2;
hwhv = 74e-2;
mwhv = 14.75e-2;
Awhv = wwhv * hwhv;

# LV winding dimensions (all phases left/right are identical)
wwlv = 2e-2;
hwlv = 74e-2;
mwlv = 11.25e-2;
Awlv = wwlv * hwlv;

# Calculate current density in the windings
Jp = Np * Ip / Awhv;
Js = Ns * Is / Awlv;

# Source current density J
# One term for each of the windings, with a positive and negative part
# Note the phase shift between the phases
sourcefunction(group_id) = Jp * exp(-1im * 2pi/3) * (1 * (group_id==3) - 1 * (group_id==4)) + 
                           Jp * (1 * (group_id==5) - 1 * (group_id==6)) + 
                           Jp * exp(1im * 2pi/3) * (1 * (group_id==7) - 1 * (group_id==8)) + 
                           Js * exp(-1im * 2pi/3) * (1 * (group_id==9) - 1 * (group_id==10)) +
                           Js * (1 * (group_id==11) - 1 * (group_id==12)) + 
                           Js * exp(1im * 2pi/3) * (1 * (group_id==13) - 1 * (group_id==14));
sourceperelement = map(sourcefunction, mesh_data.e_group);

# Relative permeability model
mu0 = 4e-7 * pi;
mur = 2500;       # Relative permeability of the core
reluctivityfunction(group_id) = (1 / mu0) + (1/(mu0*mur) - 1/mu0) * (group_id == 2);
reluctivityperelement = map(reluctivityfunction, mesh_data.e_group);

# Conductivity
conductivityfunction(group_id) = 1e-10 + (0.1 - 1e-10) * (group_id == 2);
conductivityperelement = map(conductivityfunction, mesh_data.e_group);

# Handle the boundary conditions
bnd_node_ids, _ = gmsh.model.mesh.getNodesForPhysicalGroup(1, 1);

elapsed = round((time_ns() - start)/10^9, digits=2)
println(" ✓ Initial parameters set ("*string(elapsed)*" seconds)                               ")

# Calculate the vector potential
u = fem(mesh_data, sourceperelement, reluctivityperelement, conductivityperelement, omega, bnd_node_ids);

# Post-process for magnetic field and current density
Bx, By, B, Hx, Hy, H, mag_energy = post_process(mesh_data, u, reluctivityperelement);

print(" ▸ Saving Plots .... \r")
start = time_ns()
# Contour plot of the magnetic flux density
plot_surface_tri_1e(norm.(B), "stedin_transformer_B_plot_with_eddy", gmsh)
# Contour plot of the magnetic field strength
plot_surface_tri_1e(norm.(H), "stedin_transformer_H_plot_with_eddy", gmsh)
# Contour plot of the magnetic energy
plot_surface_tri_1e(norm.(mag_energy), "stedin_transformer_mag_energy_plot_with_eddy", gmsh)
elapsed = round((time_ns() - start)/10^9, digits=2)
println(" ✓ Plots saved ("*string(elapsed)*" seconds)                               ")

print(" ▸ Saving VTK file .... \r")
start = time_ns()
# Save as VTK file for Paraview visualization
save_vtk(mesh_data, norm.(u), norm.(B), norm.(H), norm.(mag_energy), reluctivityperelement, "stedin_transformer_with_eddy")
elapsed = round((time_ns() - start)/10^9, digits=2)
println(" ✓ VTK file saved ("*string(elapsed)*" seconds)                               ")

gmsh.finalize()

Info    : Reading '../../../../Geometry/mesh/stedin_transformer.msh'...
Info    : 168 entities
Info    : 10385 nodes
Info    : 20768 elements
Info    : Done reading '../../../../Geometry/mesh/stedin_transformer.msh'
 ✓ Mesh file loaded (0.14 seconds)                               
 ✓ Mesh data generated (8.78 seconds)                               
 ✓ Initial parameters set (0.5 seconds)                               
 ✓ Matrices constructed (2.63 seconds)                               
 ✓ Solution computed (0.87 seconds)                               
 ✓ Post processing variables computed (2.08 seconds)                               
 ✓ Plots saved (49.94 seconds)                               
 ✓ VTK file saved (4.52 seconds)                               


### STEDIN Transformer : Magnetic field density
<img src="../img/stedin_transformer_B_plot_with_eddy.png">

### STEDIN Transformer : Magnetic field strength
<img src="../img/stedin_transformer_H_plot_with_eddy.png">

### STEDIN Transformer : Magnetic energy
<img src="../img/stedin_transformer_mag_energy_plot_with_eddy.png">

In [3]:
display(maximum(norm.(B)))

31.46668827377668

In [4]:
display(norm.(B[7051]))

13.943895348550836